# Deploying Machine Learning Models using kubectl
This demo shows how you can interact directly with kubernetes using kubectl to create and manage runtime machine learning models.
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

Ensure custom resource validation is activated

In [ ]:
!minikube start --feature-gates=CustomResourceValidation=true

Install Helm

In [ ]:
!helm init

Label the node to allow load testing to run on it

In [ ]:
!kubectl label nodes `kubectl get nodes -o jsonpath='{.items[0].metadata.name}'` role=locust --overwrite

## Set up REST and gRPC methods

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
from proto import prediction_pb2
from proto import prediction_pb2_grpc
import grpc

def get_token():
    payload = {'grant_type': 'client_credentials'}
    response = requests.post(
                "http://192.168.99.100:30032/oauth/token",
                auth=HTTPBasicAuth('oauth-key', 'oauth-secret'),
                data=payload)
    token =  response.json()["access_token"]
    return token

def rest_request():
    token = get_token()
    headers = {'Authorization': 'Bearer '+token}
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://192.168.99.100:30032/api/v0.1/predictions",
                headers=headers,
                json=payload)
    print response.text
    
def grpc_request():
    token = get_token()
    datadef = prediction_pb2.DefaultData(
            names = ["a","b"],
            tensor = prediction_pb2.Tensor(
                shape = [3,2],
                values = [1.0,1.0,2.0,3.0,4.0,5.0]
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel("192.168.99.100:30033")
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('oauth_token', token)]
    response = stub.Predict(request=request,metadata=metadata)
    print response


## Start seldon-core

In [ ]:
!helm install ../../helm-charts/seldon-core --name seldon-core \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false \
    --set cluster_manager.image.tag=0.3-SNAPSHOT \
    --set apife.image.tag=0.1-SNAPSHOT \
    --set engine.image.tag=0.2-SNAPSHOT

In [ ]:
!helm status seldon-core

# Integrating with Kubernetes API

## Validation

Using OpenAPI Schema certain basic validation can be done before the custom resource is accepted.

In [ ]:
!kubectl create -f resources/model_invalid1.json

For the more complex cases, e.g. checking if the graph predictive unit names for models each have an associated container in the pod spec, we need to check inside the custom resource operator and add a FAILED status.

In [ ]:
!kubectl create -f resources/model_invalid2.json

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

In [ ]:
!kubectl delete -f resources/model_invalid2.json

## Normal Operation

In [ ]:
!cat resources/model.json

## Create Seldon Deployment

In [ ]:
!kubectl apply -f resources/model.json

In [ ]:
!kubectl get seldondeployments

In [ ]:
!kubectl describe seldondeployments seldon-deployment-example 

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

## Get predictions

#### REST Request

In [ ]:
rest_request()

#### gRPC Request

In [ ]:
grpc_request()

## Update deployment with canary

In [ ]:
!cat resources/model_with_canary.json

In [ ]:
!kubectl apply -f resources/model_with_canary.json

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

#### REST Request

In [ ]:
rest_request()

#### gRPC request

In [ ]:
grpc_request()

## Load test

In [ ]:
!helm install ../../helm-charts/seldon-core-loadtesting --name loadtest  \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret 

# Tear down

In [ ]:
!helm delete loadtest --purge

In [ ]:
!kubectl delete -f resources/model_with_canary.json

In [ ]:
!helm delete seldon-core --purge